In [1]:
!pip install gradio langchain langchain-groq langchain-community chromadb pypdf sentence-transformers google-search-results lark -q


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord Ram?")
print(result.content)

Lord Rama is a major deity in Hinduism and is considered one of the most important figures in Hindu mythology. He is the seventh avatar (incarnation) of the god Vishnu and is revered as a symbol of righteousness, duty, and virtue.

According to the Hindu epic, the Ramayana, Lord Rama was a king of Ayodhya, a city in ancient India. He was born to King Dasharatha and Queen Kausalya, and was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna.

The story of Lord Rama is as follows:

* Lord Rama was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her own son, Bharata, to become the king.
* During his exile, Lord Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).
* Lord Rama, along with his brother Lakshmana and the monkey god Hanuman, embarked on a journey to rescue Sita and defeat Ravana.
* After a long and arduous battle, Lord Rama defeated Ravana and rescued Sita, and

In [ ]:
create_vector_db()

✅ ChromaDB created and data saved


 **With out speech**

In [2]:
import os
import gradio as gr
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

# ------------------ Initialize LLM ------------------ #
def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",  # 🔑 replace with your Groq API key
        model_name="llama-3.3-70b-versatile"
    )
    return llm

# ------------------ Create / Load ChromaDB ------------------ #
def create_vector_db(db_path):
    loader = DirectoryLoader("./data/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_path)
    vector_db.persist()
    print("✅ ChromaDB created and data saved")
    return vector_db

def load_vector_db(db_path):
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    print("📂 Loaded existing ChromaDB")
    return vector_db

# ------------------ Add Manual Text ------------------ #
def add_manual_text_to_db(user_text, vector_db):
    if not user_text.strip():
        return "⚠️ Please enter valid text"

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.create_documents([user_text])

    vector_db.add_documents(docs)
    vector_db.persist()
    return "✅ Your personal note was added to the knowledge base!"

# ------------------ Setup QA Chain ------------------ #
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()

    prompt_template = """
You are a personal assistant chatbot.
Answer using the retrieved context from the user's knowledge base if relevant.
If no relevant information is found, indicate that you do not know and use the web search tool.

Context: {context}
User: {question}
Chatbot:"""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# ------------------ Web Search Tool ------------------ #
def create_web_search_tool():
    search = SerpAPIWrapper(serpapi_api_key="8cffa2d32c77a91301618011fdeb91b2acd827b46e30eea10656597c2272e534")  # 🔑 replace with your SerpAPI key
    search_tool = Tool(
        name="WebSearch",
        func=search.run,
        description="Use this tool to answer questions about current events or facts."
    )
    return search_tool

# ------------------ Hybrid QA with Agent ------------------ #
def hybrid_qa(user_input):
    docs = vector_db.as_retriever().get_relevant_documents(user_input)
    if docs:
        return qa_chain.run(user_input)
    else:
        return agent.run(user_input)

# ------------------ Initialization ------------------ #
print("🚀 Initializing Personal RAG Chatbot with Web Search...")

llm = initialize_llm()

# Safe absolute path for ChromaDB
db_path = os.path.abspath("chroma_db")
os.makedirs("./data", exist_ok=True)
os.makedirs(db_path, exist_ok=True)

# Load or Create DB
if len(os.listdir(db_path)) == 0:   # if empty, build from scratch
    vector_db = create_vector_db(db_path)
else:
    vector_db = load_vector_db(db_path)

qa_chain = setup_qa_chain(vector_db, llm)
web_search = create_web_search_tool()

tools = [web_search]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

# ------------------ Gradio Functions ------------------ #
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "⚠️ Please provide a valid input"
    return hybrid_qa(user_input)

def add_knowledge(user_text):
    return add_manual_text_to_db(user_text, vector_db)

# ------------------ Gradio UI ------------------ #
with gr.Blocks(theme="Respair/Shiki@1.2.1") as app:
    with gr.Tab("💬 Chat"):
        gr.ChatInterface(
            fn=chatbot_response,
            title="🤖 Personal RAG Chatbot",
            description="Ask anything! I’ll use your personal knowledge base first, then search the web for current info."
        )

    with gr.Tab("➕ Add Knowledge"):
        user_input = gr.Textbox(lines=4, label="Enter your note, friend’s info, or any text")
        add_btn = gr.Button("Add to Knowledge Base")
        output = gr.Textbox(label="Status")

        add_btn.click(fn=add_knowledge, inputs=user_input, outputs=output)

app.launch()

🚀 Initializing Personal RAG Chatbot with Web Search...


/tmp/ipython-input-3025777792.py:31: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mod

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

/tmp/ipython-input-3025777792.py:33: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()
/tmp/ipython-input-3025777792.py:119: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


✅ ChromaDB created and data saved


theme_schema%401.2.1.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d6b2c01006635bda57.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**With some sts**

In [5]:
import os
import gradio as gr
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from groq import Groq

# ------------------ Groq Client ------------------ #
client = Groq(api_key="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")

# ------------------ Initialize LLM ------------------ #
def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )

# ------------------ Create / Load ChromaDB ------------------ #
def create_vector_db(db_path):
    loader = DirectoryLoader("./data/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_path)
    vector_db.persist()
    print("✅ ChromaDB created and data saved")
    return vector_db

def load_vector_db(db_path):
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    print("📂 Loaded existing ChromaDB")
    return vector_db

# ------------------ Add Manual Text ------------------ #
def add_manual_text_to_db(user_text, vector_db):
    if not user_text.strip():
        return "⚠️ Please enter valid text"
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.create_documents([user_text])
    vector_db.add_documents(docs)
    vector_db.persist()
    return "✅ Your personal note was added to the knowledge base!"

# ------------------ Setup QA Chain ------------------ #
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """
You are a personal assistant chatbot.
Answer using the retrieved context from the user's knowledge base if relevant.
If no relevant information is found, indicate that you do not know and use the web search tool.

Context: {context}
User: {question}
Chatbot:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# ------------------ Web Search Tool ------------------ #
def create_web_search_tool():
    search = SerpAPIWrapper(serpapi_api_key="8cffa2d32c77a91301618011fdeb91b2acd827b46e30eea10656597c2272e534")  # Replace with your SerpAPI key
    search_tool = Tool(
        name="WebSearch",
        func=search.run,
        description="Use this tool to answer questions about current events or facts."
    )
    return search_tool

# ------------------ Hybrid QA ------------------ #
def hybrid_qa(user_input):
    docs = vector_db.as_retriever().get_relevant_documents(user_input)
    if docs:
        return qa_chain.run(user_input)
    else:
        return agent.run(user_input)

# ------------------ TTS / STT / STS ------------------ #
def text_to_speech(text):
    if not text.strip():
        return None
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=text,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

def speech_to_text(audio_file):
    if not audio_file:
        return "⚠️ Please provide an audio input"
    with open(audio_file, "rb") as f:
        stream = client.audio.translations.create(
            file=(audio_file, f.read()),
            model="whisper-large-v3",
            temperature=0,
            response_format="json",
            prompt="transcribe the speech"
        )
    return stream.text

def speech_to_speech(audio_file):
    text = speech_to_text(audio_file)
    answer = hybrid_qa(text)
    audio_path = text_to_speech(answer)
    return audio_path

# ------------------ Chatbot Response ------------------ #
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "⚠️ Please provide a valid input"
    return hybrid_qa(user_input)

def add_knowledge(user_text):
    return add_manual_text_to_db(user_text, vector_db)

# ------------------ Initialization ------------------ #
print("🚀 Initializing Personal RAG Chatbot with Web Search...")
llm = initialize_llm()
db_path = os.path.abspath("chroma_db")
os.makedirs("./data", exist_ok=True)
os.makedirs(db_path, exist_ok=True)

if len(os.listdir(db_path)) == 0:
    vector_db = create_vector_db(db_path)
else:
    vector_db = load_vector_db(db_path)

qa_chain = setup_qa_chain(vector_db, llm)
tools = [create_web_search_tool()]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

# ------------------ Gradio UI ------------------ #
with gr.Blocks(theme="Respair/Shiki@1.2.1") as app:
    with gr.Tab("💬 Chat"):
        # Chat interface
        gr.ChatInterface(
            fn=chatbot_response,
            title="🤖 Personal RAG Chatbot",
            description="Ask anything! I’ll use your personal knowledge base first, then search the web for current info.",
            type="messages"
        )

        # TTS
        tts_input = gr.Textbox(label="Text to Speech")
        tts_output = gr.Audio(label="Output Speech")
        tts_btn = gr.Button("Convert to Speech")
        tts_btn.click(fn=text_to_speech, inputs=tts_input, outputs=tts_output)

        # STT
        stt_input = gr.Audio(label="Speak to Chatbot", type="filepath")  # microphone recording enabled automatically
        stt_output = gr.Textbox(label="Transcribed Text")
        stt_btn = gr.Button("Transcribe Speech")
        stt_btn.click(fn=speech_to_text, inputs=stt_input, outputs=stt_output)

        # STS
        sts_input = gr.Audio(label="Speak and hear response", type="filepath")
        sts_output = gr.Audio(label="Chatbot Response")
        sts_btn = gr.Button("Speak & Hear Response")
        sts_btn.click(fn=speech_to_speech, inputs=sts_input, outputs=sts_output)

    with gr.Tab("➕ Add Knowledge"):
        user_input = gr.Textbox(lines=4, label="Enter your note, friend’s info, or any text")
        add_btn = gr.Button("Add to Knowledge Base")
        output = gr.Textbox(label="Status")
        add_btn.click(fn=add_knowledge, inputs=user_input, outputs=output)

app.launch()


🚀 Initializing Personal RAG Chatbot with Web Search...
📂 Loaded existing ChromaDB
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3b0903750902358a18.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**With all TTS STT STS**

In [6]:
import os
import gradio as gr
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from groq import Groq

# ------------------ Groq Client ------------------ #
client = Groq(api_key="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")

# ------------------ Initialize LLM ------------------ #
def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )

# ------------------ Create / Load ChromaDB ------------------ #
def create_vector_db(db_path):
    loader = DirectoryLoader("./data/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_path)
    vector_db.persist()
    print("✅ ChromaDB created and data saved")
    return vector_db

def load_vector_db(db_path):
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    print("📂 Loaded existing ChromaDB")
    return vector_db

# ------------------ Add Manual Text ------------------ #
def add_manual_text_to_db(user_text, vector_db):
    if not user_text.strip():
        return "⚠️ Please enter valid text"
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.create_documents([user_text])
    vector_db.add_documents(docs)
    vector_db.persist()
    return "✅ Your personal note was added to the knowledge base!"

# ------------------ Setup QA Chain ------------------ #
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """
You are a personal assistant chatbot.
Answer using the retrieved context from the user's knowledge base if relevant.
If no relevant information is found, indicate that you do not know and use the web search tool.

Context: {context}
User: {question}
Chatbot:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# ------------------ Web Search Tool ------------------ #
def create_web_search_tool():
    search = SerpAPIWrapper(serpapi_api_key="8cffa2d32c77a91301618011fdeb91b2acd827b46e30eea10656597c2272e534")  # Replace with your SerpAPI key
    search_tool = Tool(
        name="WebSearch",
        func=search.run,
        description="Use this tool to answer questions about current events or facts."
    )
    return search_tool

# ------------------ Hybrid QA ------------------ #
def hybrid_qa(user_input):
    docs = vector_db.as_retriever().get_relevant_documents(user_input)
    if docs:
        return qa_chain.run(user_input)
    else:
        return agent.run(user_input)

# ------------------ TTS / STT / STS ------------------ #
def text_to_speech_chatbot(user_input):
    answer = hybrid_qa(user_input)  # Get chatbot answer
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=answer,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

def speech_to_text_chatbot(audio_file):
    if not audio_file:
        return "⚠️ Please provide an audio input"

    # Step 1: Transcribe audio
    with open(audio_file, "rb") as f:
        stream = client.audio.translations.create(
            file=(audio_file, f.read()),
            model="whisper-large-v3",
            temperature=0,
            response_format="json",
            prompt="transcribe the speech"
        )
    transcribed_text = stream.text

    # Step 2: Send transcribed text to chatbot
    answer = hybrid_qa(transcribed_text)
    return answer

def speech_to_speech(audio_file):
    # STS: Audio → Chatbot → Audio
    text = speech_to_text_chatbot(audio_file)  # Already returns chatbot answer
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=text,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

# ------------------ Chatbot Response ------------------ #
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "⚠️ Please provide a valid input"
    return hybrid_qa(user_input)

def add_knowledge(user_text):
    return add_manual_text_to_db(user_text, vector_db)

# ------------------ Initialization ------------------ #
print("🚀 Initializing Personal RAG Chatbot with Web Search...")
llm = initialize_llm()
db_path = os.path.abspath("chroma_db")
os.makedirs("./data", exist_ok=True)
os.makedirs(db_path, exist_ok=True)

if len(os.listdir(db_path)) == 0:
    vector_db = create_vector_db(db_path)
else:
    vector_db = load_vector_db(db_path)

qa_chain = setup_qa_chain(vector_db, llm)
tools = [create_web_search_tool()]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

# ------------------ Gradio UI ------------------ #
with gr.Blocks(theme="Respair/Shiki@1.2.1") as app:
    with gr.Tab("💬 Chat"):
        gr.ChatInterface(
            fn=chatbot_response,
            title="🤖 Personal RAG Chatbot",
            description="Ask anything! I’ll use your personal knowledge base first, then search the web for current info.",
            type="messages"
        )

        # TTS (Text → Chatbot → Speech)
        tts_input = gr.Textbox(label="Ask the bot (Text-to-Speech)")
        tts_output = gr.Audio(label="Bot Answer (Speech)")
        tts_btn = gr.Button("Speak Answer")
        tts_btn.click(fn=text_to_speech_chatbot, inputs=tts_input, outputs=tts_output)

        # STT (Speech → Chatbot → Text)
        stt_input = gr.Audio(label="Ask the bot (Speech-to-Text)", type="filepath")
        stt_output = gr.Textbox(label="Bot Answer (Text)")
        stt_btn = gr.Button("Get Answer")
        stt_btn.click(fn=speech_to_text_chatbot, inputs=stt_input, outputs=stt_output)

        # STS (Speech → Chatbot → Speech)
        sts_input = gr.Audio(label="Ask the bot (Speech-to-Speech)", type="filepath")
        sts_output = gr.Audio(label="Bot Answer (Speech)")
        sts_btn = gr.Button("Speak & Hear Answer")
        sts_btn.click(fn=speech_to_speech, inputs=sts_input, outputs=sts_output)

    with gr.Tab("➕ Add Knowledge"):
        user_input = gr.Textbox(lines=4, label="Enter your note, friend’s info, or any text")
        add_btn = gr.Button("Add to Knowledge Base")
        output = gr.Textbox(label="Status")
        add_btn.click(fn=add_knowledge, inputs=user_input, outputs=output)

app.launch()


🚀 Initializing Personal RAG Chatbot with Web Search...
📂 Loaded existing ChromaDB
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1f48ec65cc1d14335b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**With all and web search using Groq**

In [7]:
import os
import gradio as gr
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from groq import Groq

# ------------------ Groq Client ------------------ #
client = Groq(api_key="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")

# ------------------ Initialize LLM ------------------ #
def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key=client.api_key,
        model_name="llama-3.3-70b-versatile"
    )

# ------------------ Groq “Web Search” Tool ------------------ #
def groq_web_search(query):
    """
    Use Groq 'compound' model to answer open-domain questions.
    """
    response = client.chat.completions.create(
        model="groq/compound",
        messages=[{"role": "user", "content": query}],
        temperature=0.0
    )
    return response.choices[0].message.content

def create_groq_search_tool():
    return Tool(
        name="GroqWebSearch",
        func=groq_web_search,
        description="Use this tool to answer open-domain questions or current facts."
    )

# ------------------ Create / Load ChromaDB ------------------ #
def create_vector_db(db_path):
    loader = DirectoryLoader("./data/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_path)
    vector_db.persist()
    print("✅ ChromaDB created and data saved")
    return vector_db

def load_vector_db(db_path):
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    print("📂 Loaded existing ChromaDB")
    return vector_db

# ------------------ Add Manual Text ------------------ #
def add_manual_text_to_db(user_text, vector_db):
    if not user_text.strip():
        return "⚠️ Please enter valid text"
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.create_documents([user_text])
    vector_db.add_documents(docs)
    vector_db.persist()
    return "✅ Your personal note was added to the knowledge base!"

# ------------------ Setup QA Chain ------------------ #
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """
You are a personal assistant chatbot.
Answer using the retrieved context from the user's knowledge base if relevant.
If no relevant information is found, indicate that you do not know and use the web search tool.

Context: {context}
User: {question}
Chatbot:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# ------------------ Hybrid QA ------------------ #
def hybrid_qa(user_input):
    docs = vector_db.as_retriever().get_relevant_documents(user_input)
    if docs:
        return qa_chain.run(user_input)
    else:
        # If no relevant docs, use Groq compound model as "web search"
        return groq_web_search(user_input)

# ------------------ TTS / STT / STS ------------------ #
def text_to_speech_chatbot(user_input):
    answer = hybrid_qa(user_input)
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=answer,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

def speech_to_text_chatbot(audio_file):
    if not audio_file:
        return "⚠️ Please provide an audio input"

    with open(audio_file, "rb") as f:
        stream = client.audio.translations.create(
            file=(audio_file, f.read()),
            model="whisper-large-v3",
            temperature=0,
            response_format="json",
            prompt="transcribe the speech"
        )
    transcribed_text = stream.text
    answer = hybrid_qa(transcribed_text)
    return answer

def speech_to_speech(audio_file):
    text = speech_to_text_chatbot(audio_file)
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=text,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

# ------------------ Chatbot Response ------------------ #
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "⚠️ Please provide a valid input"
    return hybrid_qa(user_input)

def add_knowledge(user_text):
    return add_manual_text_to_db(user_text, vector_db)

# ------------------ Initialization ------------------ #
print("🚀 Initializing Personal RAG Chatbot with Groq Compound for web search...")
llm = initialize_llm()
db_path = os.path.abspath("chroma_db")
os.makedirs("./data", exist_ok=True)
os.makedirs(db_path, exist_ok=True)

if len(os.listdir(db_path)) == 0:
    vector_db = create_vector_db(db_path)
else:
    vector_db = load_vector_db(db_path)

qa_chain = setup_qa_chain(vector_db, llm)

# ------------------ Gradio UI ------------------ #
with gr.Blocks(theme="Respair/Shiki@1.2.1") as app:
    with gr.Tab("💬 Chat"):
        gr.ChatInterface(
            fn=chatbot_response,
            title="🤖 Personal RAG Chatbot",
            description="Ask anything! I’ll use your personal knowledge base first, then Groq compound for open questions.",
            type="messages"
        )

        # TTS (Text → Chatbot → Speech)
        tts_input = gr.Textbox(label="Ask the bot (Text-to-Speech)")
        tts_output = gr.Audio(label="Bot Answer (Speech)")
        tts_btn = gr.Button("Speak Answer")
        tts_btn.click(fn=text_to_speech_chatbot, inputs=tts_input, outputs=tts_output)

        # STT (Speech → Chatbot → Text)
        stt_input = gr.Audio(label="Ask the bot (Speech-to-Text)", type="filepath")
        stt_output = gr.Textbox(label="Bot Answer (Text)")
        stt_btn = gr.Button("Get Answer")
        stt_btn.click(fn=speech_to_text_chatbot, inputs=stt_input, outputs=stt_output)

        # STS (Speech → Chatbot → Speech)
        sts_input = gr.Audio(label="Ask the bot (Speech-to-Speech)", type="filepath")
        sts_output = gr.Audio(label="Bot Answer (Speech)")
        sts_btn = gr.Button("Speak & Hear Answer")
        sts_btn.click(fn=speech_to_speech, inputs=sts_input, outputs=sts_output)

    with gr.Tab("➕ Add Knowledge"):
        user_input = gr.Textbox(lines=4, label="Enter your note, friend’s info, or any text")
        add_btn = gr.Button("Add to Knowledge Base")
        output = gr.Textbox(label="Status")
        add_btn.click(fn=add_knowledge, inputs=user_input, outputs=output)

app.launch()


🚀 Initializing Personal RAG Chatbot with Groq Compound for web search...
📂 Loaded existing ChromaDB
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bd23540bf99705e6bb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**With all and Browse Search**

In [8]:
import os
import gradio as gr
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from groq import Groq

# ------------------ Groq Client ------------------ #
client = Groq(api_key="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")

# ------------------ Initialize LLM ------------------ #
def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key=client.api_key,
        model_name="llama-3.3-70b-versatile"
    )

# ------------------ Groq Browse Tool ------------------ #
def groq_browse(query):
    """
    Use Groq browse model `openai/gpt-oss-120b` to answer open-domain questions.
    """
    response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[{"role": "user", "content": query}],
        temperature=0.0
    )
    return response.choices[0].message.content

def create_groq_browse_tool():
    return Tool(
        name="GroqBrowse",
        func=groq_browse,
        description="Use this tool to answer open-domain questions via the browse model."
    )

# ------------------ Create / Load ChromaDB ------------------ #
def create_vector_db(db_path):
    loader = DirectoryLoader("./data/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_path)
    vector_db.persist()
    print("✅ ChromaDB created and data saved")
    return vector_db

def load_vector_db(db_path):
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    print("📂 Loaded existing ChromaDB")
    return vector_db

# ------------------ Add Manual Text ------------------ #
def add_manual_text_to_db(user_text, vector_db):
    if not user_text.strip():
        return "⚠️ Please enter valid text"
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.create_documents([user_text])
    vector_db.add_documents(docs)
    vector_db.persist()
    return "✅ Your personal note was added to the knowledge base!"

# ------------------ Setup QA Chain ------------------ #
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """
You are a personal assistant chatbot.
Answer using the retrieved context from the user's knowledge base if relevant.
If no relevant information is found, indicate that you do not know and use the browse tool.

Context: {context}
User: {question}
Chatbot:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# ------------------ Hybrid QA ------------------ #
def hybrid_qa(user_input):
    docs = vector_db.as_retriever().get_relevant_documents(user_input)
    if docs:
        return qa_chain.run(user_input)
    else:
        # If no relevant docs, use Groq browse model
        return groq_browse(user_input)

# ------------------ TTS / STT / STS ------------------ #
def text_to_speech_chatbot(user_input):
    answer = hybrid_qa(user_input)
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=answer,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

def speech_to_text_chatbot(audio_file):
    if not audio_file:
        return "⚠️ Please provide an audio input"

    with open(audio_file, "rb") as f:
        stream = client.audio.translations.create(
            file=(audio_file, f.read()),
            model="whisper-large-v3",
            temperature=0,
            response_format="json",
            prompt="transcribe the speech"
        )
    transcribed_text = stream.text
    answer = hybrid_qa(transcribed_text)
    return answer

def speech_to_speech(audio_file):
    text = speech_to_text_chatbot(audio_file)
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=text,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

# ------------------ Chatbot Response ------------------ #
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "⚠️ Please provide a valid input"
    return hybrid_qa(user_input)

def add_knowledge(user_text):
    return add_manual_text_to_db(user_text, vector_db)

# ------------------ Initialization ------------------ #
print("🚀 Initializing Personal RAG Chatbot with Groq Browse Model...")
llm = initialize_llm()
db_path = os.path.abspath("chroma_db")
os.makedirs("./data", exist_ok=True)
os.makedirs(db_path, exist_ok=True)

if len(os.listdir(db_path)) == 0:
    vector_db = create_vector_db(db_path)
else:
    vector_db = load_vector_db(db_path)

qa_chain = setup_qa_chain(vector_db, llm)

# ------------------ Gradio UI ------------------ #
with gr.Blocks(theme="Respair/Shiki@1.2.1") as app:
    with gr.Tab("💬 Chat"):
        gr.ChatInterface(
            fn=chatbot_response,
            title="🤖 Personal RAG Chatbot",
            description="Ask anything! I’ll use your personal knowledge base first, then Groq browse model for other queries.",
            type="messages"
        )

        # TTS (Text → Chatbot → Speech)
        tts_input = gr.Textbox(label="Ask the bot (Text-to-Speech)")
        tts_output = gr.Audio(label="Bot Answer (Speech)")
        tts_btn = gr.Button("Speak Answer")
        tts_btn.click(fn=text_to_speech_chatbot, inputs=tts_input, outputs=tts_output)

        # STT (Speech → Chatbot → Text)
        stt_input = gr.Audio(label="Ask the bot (Speech-to-Text)", type="filepath")
        stt_output = gr.Textbox(label="Bot Answer (Text)")
        stt_btn = gr.Button("Get Answer")
        stt_btn.click(fn=speech_to_text_chatbot, inputs=stt_input, outputs=stt_output)

        # STS (Speech → Chatbot → Speech)
        sts_input = gr.Audio(label="Ask the bot (Speech-to-Speech)", type="filepath")
        sts_output = gr.Audio(label="Bot Answer (Speech)")
        sts_btn = gr.Button("Speak & Hear Answer")
        sts_btn.click(fn=speech_to_speech, inputs=sts_input, outputs=sts_output)

    with gr.Tab("➕ Add Knowledge"):
        user_input = gr.Textbox(lines=4, label="Enter your note, friend’s info, or any text")
        add_btn = gr.Button("Add to Knowledge Base")
        output = gr.Textbox(label="Status")
        add_btn.click(fn=add_knowledge, inputs=user_input, outputs=output)

app.launch()


🚀 Initializing Personal RAG Chatbot with Groq Browse Model...
📂 Loaded existing ChromaDB
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d71caa4f246d23b39b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**With all and SerpAPI web search**

In [9]:
import os
import gradio as gr
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from groq import Groq

# ------------------ Groq Client ------------------ #
client = Groq(api_key="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")

# ------------------ Initialize LLM ------------------ #
def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key=client.api_key,
        model_name="llama-3.3-70b-versatile"
    )

# ------------------ Create / Load ChromaDB ------------------ #
def create_vector_db(db_path):
    loader = DirectoryLoader("./data/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_path)
    vector_db.persist()
    print("✅ ChromaDB created and data saved")
    return vector_db

def load_vector_db(db_path):
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    print("📂 Loaded existing ChromaDB")
    return vector_db

# ------------------ Add Manual Text ------------------ #
def add_manual_text_to_db(user_text, vector_db):
    if not user_text.strip():
        return "⚠️ Please enter valid text"
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.create_documents([user_text])
    vector_db.add_documents(docs)
    vector_db.persist()
    return "✅ Your personal note was added to the knowledge base!"

# ------------------ Setup QA Chain ------------------ #
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """
You are a personal assistant chatbot.
Answer using the retrieved context from the user's knowledge base if relevant.
If no relevant information is found, indicate that you do not know and use the fallback tool.

Context: {context}
User: {question}
Chatbot:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# ------------------ Groq Browse Tool ------------------ #
def groq_browse(query):
    response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[{"role": "user", "content": query}],
        temperature=0.0
    )
    return response.choices[0].message.content

def create_groq_browse_tool():
    return Tool(
        name="GroqBrowse",
        func=groq_browse,
        description="Use this tool to answer open-domain questions via the Groq browse model."
    )

# ------------------ SerpAPI Live Search Tool ------------------ #
def create_live_search_tool():
    search = SerpAPIWrapper(serpapi_api_key="8cffa2d32c77a91301618011fdeb91b2acd827b46e30eea10656597c2272e534")  # Replace with your SerpAPI key
    return Tool(
        name="LiveSearch",
        func=search.run,
        description="Use this tool to answer live queries like sports scores, news, etc."
    )

# ------------------ Hybrid QA ------------------ #
def hybrid_qa(user_input):
    docs = vector_db.as_retriever().get_relevant_documents(user_input)
    if docs:
        return qa_chain.run(user_input)
    else:
        # Step 1: Try Groq browse
        browse_answer = groq_browse(user_input)
        if browse_answer and "I don't have" not in browse_answer:
            return browse_answer
        # Step 2: Use live search + LLM extraction
        search_results = live_search_tool.run(user_input)
        prompt = PromptTemplate(
            template="""
Answer the following question based on the search results:

Search Results:
{results}

Question: {question}
Answer:
""",
            input_variables=["results", "question"]
        )
        chain = LLMChain(llm=llm, prompt=prompt)
        return chain.run(results=search_results, question=user_input)

# ------------------ TTS / STT / STS ------------------ #
def text_to_speech_chatbot(user_input):
    answer = hybrid_qa(user_input)
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=answer,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

def speech_to_text_chatbot(audio_file):
    if not audio_file:
        return "⚠️ Please provide an audio input"
    with open(audio_file, "rb") as f:
        stream = client.audio.translations.create(
            file=(audio_file, f.read()),
            model="whisper-large-v3",
            temperature=0,
            response_format="json",
            prompt="transcribe the speech"
        )
    transcribed_text = stream.text
    answer = hybrid_qa(transcribed_text)
    return answer

def speech_to_speech(audio_file):
    text = speech_to_text_chatbot(audio_file)
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=text,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

# ------------------ Chatbot Response ------------------ #
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "⚠️ Please provide a valid input"
    return hybrid_qa(user_input)

def add_knowledge(user_text):
    return add_manual_text_to_db(user_text, vector_db)

# ------------------ Initialization ------------------ #
print("🚀 Initializing Personal RAG Chatbot with Groq Browse + Live Search...")
llm = initialize_llm()
db_path = os.path.abspath("chroma_db")
os.makedirs("./data", exist_ok=True)
os.makedirs(db_path, exist_ok=True)

if len(os.listdir(db_path)) == 0:
    vector_db = create_vector_db(db_path)
else:
    vector_db = load_vector_db(db_path)

qa_chain = setup_qa_chain(vector_db, llm)
live_search_tool = create_live_search_tool()

# ------------------ Gradio UI ------------------ #
with gr.Blocks(theme="Respair/Shiki@1.2.1") as app:
    with gr.Tab("💬 Chat"):
        gr.ChatInterface(
            fn=chatbot_response,
            title="🤖 Personal RAG Chatbot",
            description="Ask anything! I’ll use your personal knowledge base first, then Groq browse model, then live search for real-time info.",
            type="messages"
        )

        # TTS
        tts_input = gr.Textbox(label="Ask the bot (Text-to-Speech)")
        tts_output = gr.Audio(label="Bot Answer (Speech)")
        tts_btn = gr.Button("Speak Answer")
        tts_btn.click(fn=text_to_speech_chatbot, inputs=tts_input, outputs=tts_output)

        # STT
        stt_input = gr.Audio(label="Ask the bot (Speech-to-Text)", type="filepath")
        stt_output = gr.Textbox(label="Bot Answer (Text)")
        stt_btn = gr.Button("Get Answer")
        stt_btn.click(fn=speech_to_text_chatbot, inputs=stt_input, outputs=stt_output)

        # STS
        sts_input = gr.Audio(label="Ask the bot (Speech-to-Speech)", type="filepath")
        sts_output = gr.Audio(label="Bot Answer (Speech)")
        sts_btn = gr.Button("Speak & Hear Answer")
        sts_btn.click(fn=speech_to_speech, inputs=sts_input, outputs=sts_output)

    with gr.Tab("➕ Add Knowledge"):
        user_input = gr.Textbox(lines=4, label="Enter your note, friend’s info, or any text")
        add_btn = gr.Button("Add to Knowledge Base")
        output = gr.Textbox(label="Status")
        add_btn.click(fn=add_knowledge, inputs=user_input, outputs=output)

app.launch()


🚀 Initializing Personal RAG Chatbot with Groq Browse + Live Search...
📂 Loaded existing ChromaDB
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd9f9a08612b4264f8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Continues version**

In [10]:
import os
import gradio as gr
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from groq import Groq

# ------------------ Groq Client ------------------ #
client = Groq(api_key="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")

# ------------------ Global Conversation History ------------------ #
conversation_history = []

# ------------------ Initialize LLM ------------------ #
def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key=client.api_key,
        model_name="llama-3.3-70b-versatile"
    )

# ------------------ Create / Load ChromaDB ------------------ #
def create_vector_db(db_path):
    loader = DirectoryLoader("./data/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_path)
    vector_db.persist()
    print("✅ ChromaDB created and data saved")
    return vector_db

def load_vector_db(db_path):
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    print("📂 Loaded existing ChromaDB")
    return vector_db

# ------------------ Add Manual Text ------------------ #
def add_manual_text_to_db(user_text, vector_db):
    if not user_text.strip():
        return "⚠️ Please enter valid text"
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.create_documents([user_text])
    vector_db.add_documents(docs)
    vector_db.persist()
    return "✅ Your personal note was added to the knowledge base!"

# ------------------ Setup QA Chain ------------------ #
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """
You are a personal assistant chatbot.
Answer using the retrieved context from the user's knowledge base if relevant.
If no relevant information is found, indicate that you do not know and use the fallback tool.

Context: {context}
User: {question}
Chatbot:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# ------------------ Groq Browse Tool ------------------ #
def groq_browse(query):
    response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[{"role": "user", "content": query}],
        temperature=0.0
    )
    return response.choices[0].message.content

# ------------------ SerpAPI Live Search Tool ------------------ #
def create_live_search_tool():
    search = SerpAPIWrapper(serpapi_api_key="8cffa2d32c77a91301618011fdeb91b2acd827b46e30eea10656597c2272e534")  # Replace with your SerpAPI key
    return Tool(
        name="LiveSearch",
        func=search.run,
        description="Use this tool to answer live queries like sports scores, news, etc."
    )

# ------------------ Hybrid QA with Continuous Context ------------------ #
def hybrid_qa(user_input):
    # Build conversation context
    context = "\n".join([f"User: {u}\nBot: {b}" for u, b in conversation_history])
    full_input = f"{context}\nUser: {user_input}" if context else user_input

    # Step 1: Check vector DB
    docs = vector_db.as_retriever().get_relevant_documents(full_input)
    if docs:
        answer = qa_chain.run(full_input)
    else:
        # Step 2: Groq browse
        browse_answer = groq_browse(full_input)
        if browse_answer and "I don't have" not in browse_answer:
            answer = browse_answer
        else:
            # Step 3: Live search + LLM extraction
            search_results = live_search_tool.run(user_input)
            prompt = PromptTemplate(
                template="""
Answer the following question based on the search results:

Search Results:
{results}

Question: {question}
Answer:
""",
                input_variables=["results", "question"]
            )
            chain = LLMChain(llm=llm, prompt=prompt)
            answer = chain.run(results=search_results, question=user_input)

    # Update conversation history
    conversation_history.append((user_input, answer))
    return answer

# ------------------ TTS / STT / STS ------------------ #
def text_to_speech_chatbot(user_input):
    answer = hybrid_qa(user_input)
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=answer,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

def speech_to_text_chatbot(audio_file):
    if not audio_file:
        return "⚠️ Please provide an audio input"
    with open(audio_file, "rb") as f:
        stream = client.audio.translations.create(
            file=(audio_file, f.read()),
            model="whisper-large-v3",
            temperature=0,
            response_format="json",
            prompt="transcribe the speech"
        )
    transcribed_text = stream.text
    answer = hybrid_qa(transcribed_text)
    return answer

def speech_to_speech(audio_file):
    text = speech_to_text_chatbot(audio_file)
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=text,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

# ------------------ Chatbot Response ------------------ #
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "⚠️ Please provide a valid input"
    return hybrid_qa(user_input)

def add_knowledge(user_text):
    return add_manual_text_to_db(user_text, vector_db)

# ------------------ Initialization ------------------ #
print("🚀 Initializing Personal RAG Chatbot with Continuous Context...")
llm = initialize_llm()
db_path = os.path.abspath("chroma_db")
os.makedirs("./data", exist_ok=True)
os.makedirs(db_path, exist_ok=True)

if len(os.listdir(db_path)) == 0:
    vector_db = create_vector_db(db_path)
else:
    vector_db = load_vector_db(db_path)

qa_chain = setup_qa_chain(vector_db, llm)
live_search_tool = create_live_search_tool()

# ------------------ Gradio UI ------------------ #
with gr.Blocks(theme="Respair/Shiki@1.2.1") as app:
    with gr.Tab("💬 Chat"):
        gr.ChatInterface(
            fn=chatbot_response,
            title="🤖 Personal RAG Chatbot",
            description="Ask anything! Your conversation context is remembered, and I can use Groq browse or live search for other queries.",
            type="messages"
        )

        # TTS
        tts_input = gr.Textbox(label="Ask the bot (Text-to-Speech)")
        tts_output = gr.Audio(label="Bot Answer (Speech)")
        tts_btn = gr.Button("Speak Answer")
        tts_btn.click(fn=text_to_speech_chatbot, inputs=tts_input, outputs=tts_output)

        # STT
        stt_input = gr.Audio(label="Ask the bot (Speech-to-Text)", type="filepath")
        stt_output = gr.Textbox(label="Bot Answer (Text)")
        stt_btn = gr.Button("Get Answer")
        stt_btn.click(fn=speech_to_text_chatbot, inputs=stt_input, outputs=stt_output)

        # STS
        sts_input = gr.Audio(label="Ask the bot (Speech-to-Speech)", type="filepath")
        sts_output = gr.Audio(label="Bot Answer (Speech)")
        sts_btn = gr.Button("Speak & Hear Answer")
        sts_btn.click(fn=speech_to_speech, inputs=sts_input, outputs=sts_output)

    with gr.Tab("➕ Add Knowledge"):
        user_input = gr.Textbox(lines=4, label="Enter your note, friend’s info, or any text")
        add_btn = gr.Button("Add to Knowledge Base")
        output = gr.Textbox(label="Status")
        add_btn.click(fn=add_knowledge, inputs=user_input, outputs=output)

app.launch()


🚀 Initializing Personal RAG Chatbot with Continuous Context...
📂 Loaded existing ChromaDB
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9aad175866e8b30b4f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**I think it is final**

In [11]:
import os
import gradio as gr
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from groq import Groq

# ------------------ Groq Client ------------------ #
client = Groq(api_key="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")

# ------------------ Global Conversation History ------------------ #
conversation_history = []

# ------------------ Initialize LLM ------------------ #
def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key=client.api_key,
        model_name="llama-3.3-70b-versatile"
    )

# ------------------ Create / Load ChromaDB ------------------ #
def create_vector_db(db_path):
    loader = DirectoryLoader("./data/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_path)
    vector_db.persist()
    print("✅ ChromaDB created and data saved")
    return vector_db

def load_vector_db(db_path):
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    print("📂 Loaded existing ChromaDB")
    return vector_db

# ------------------ Add Manual Text ------------------ #
def add_manual_text_to_db(user_text, vector_db):
    if not user_text.strip():
        return "⚠️ Please enter valid text"
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.create_documents([user_text])
    vector_db.add_documents(docs)
    vector_db.persist()
    return "✅ Your personal note was added to the knowledge base!"

# ------------------ Setup QA Chain ------------------ #
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """
You are a helpful personal assistant chatbot.
Answer concisely using the retrieved context from the user's knowledge base if relevant.

Context: {context}
User: {question}
Chatbot:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# ------------------ Groq Browse Tool ------------------ #
def groq_browse(query):
    response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[{"role": "user", "content": query}],
        temperature=0.0
    )
    return response.choices[0].message.content

# ------------------ SerpAPI Live Search Tool ------------------ #
def create_live_search_tool():
    search = SerpAPIWrapper(serpapi_api_key="8cffa2d32c77a91301618011fdeb91b2acd827b46e30eea10656597c2272e534")  # Replace with your SerpAPI key
    return Tool(
        name="LiveSearch",
        func=search.run,
        description="Use this tool to answer live queries like sports scores, news, etc."
    )

# ------------------ Hybrid QA ------------------ #
def hybrid_qa(user_input):
    # Keep conversation context
    context = "\n".join([f"User: {u}\nBot: {b}" for u, b in conversation_history])
    full_input = f"{context}\nUser: {user_input}" if context else user_input

    # Step 1: Check vector DB
    docs = vector_db.as_retriever().get_relevant_documents(full_input)
    if docs:
        answer = qa_chain.run(full_input)
    else:
        # Step 2: Groq browse
        browse_answer = groq_browse(full_input)
        if browse_answer:
            answer = browse_answer
        else:
            # Step 3: Live search + LLM extraction
            search_results = live_search_tool.run(user_input)
            prompt = PromptTemplate(
                template="""Answer the following question based on the search results:

Search Results:
{results}

Question: {question}
Answer:""",
                input_variables=["results", "question"]
            )
            chain = LLMChain(llm=llm, prompt=prompt)
            answer = chain.run(results=search_results, question=user_input)

    conversation_history.append((user_input, answer))
    return answer

# ------------------ TTS / STT / STS ------------------ #
def text_to_speech_chatbot(user_input):
    answer = hybrid_qa(user_input)
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=answer,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

def speech_to_text_chatbot(audio_file):
    if not audio_file:
        return "⚠️ Please provide an audio input"
    with open(audio_file, "rb") as f:
        stream = client.audio.translations.create(
            file=(audio_file, f.read()),
            model="whisper-large-v3",
            temperature=0,
            response_format="json",
            prompt="transcribe the speech"
        )
    transcribed_text = stream.text
    answer = hybrid_qa(transcribed_text)
    return answer

def speech_to_speech(audio_file):
    text = speech_to_text_chatbot(audio_file)
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=text,
        response_format="wav"
    )
    file_path = "output_speech.wav"
    response.write_to_file(file_path)
    return file_path

# ------------------ Chatbot Response ------------------ #
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "⚠️ Please provide a valid input"
    return hybrid_qa(user_input)

def add_knowledge(user_text):
    return add_manual_text_to_db(user_text, vector_db)

# ------------------ Initialization ------------------ #
print("🚀 Initializing Personal RAG Chatbot without preamble messages...")
llm = initialize_llm()
db_path = os.path.abspath("chroma_db")
os.makedirs("./data", exist_ok=True)
os.makedirs(db_path, exist_ok=True)

if len(os.listdir(db_path)) == 0:
    vector_db = create_vector_db(db_path)
else:
    vector_db = load_vector_db(db_path)

qa_chain = setup_qa_chain(vector_db, llm)
live_search_tool = create_live_search_tool()

# ------------------ Gradio UI ------------------ #
with gr.Blocks(theme="Respair/Shiki@1.2.1") as app:
    with gr.Tab("💬 Chat"):
        gr.ChatInterface(
            fn=chatbot_response,
            title="🤖 Personal RAG Chatbot",
            description="Ask anything! Answers come directly from your knowledge base, Groq browse, or live search.",
            type="messages"
        )

        # TTS
        tts_input = gr.Textbox(label="Ask the bot (Text-to-Speech)")
        tts_output = gr.Audio(label="Bot Answer (Speech)")
        tts_btn = gr.Button("Speak Answer")
        tts_btn.click(fn=text_to_speech_chatbot, inputs=tts_input, outputs=tts_output)

        # STT
        stt_input = gr.Audio(label="Ask the bot (Speech-to-Text)", type="filepath")
        stt_output = gr.Textbox(label="Bot Answer (Text)")
        stt_btn = gr.Button("Get Answer")
        stt_btn.click(fn=speech_to_text_chatbot, inputs=stt_input, outputs=stt_output)

        # STS
        sts_input = gr.Audio(label="Ask the bot (Speech-to-Speech)", type="filepath")
        sts_output = gr.Audio(label="Bot Answer (Speech)")
        sts_btn = gr.Button("Speak & Hear Answer")
        sts_btn.click(fn=speech_to_speech, inputs=sts_input, outputs=sts_output)

    with gr.Tab("➕ Add Knowledge"):
        user_input = gr.Textbox(lines=4, label="Enter your note, friend’s info, or any text")
        add_btn = gr.Button("Add to Knowledge Base")
        output = gr.Textbox(label="Status")
        add_btn.click(fn=add_knowledge, inputs=user_input, outputs=output)

app.launch()


🚀 Initializing Personal RAG Chatbot without preamble messages...
📂 Loaded existing ChromaDB
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b89b80b67b164a3480.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
